## imports

In [ ]:
import os
import sys
sys.path.append("../")

import math

import pandas as pd
import numpy as np
import datetime as dt
import re

import pymc3 as pm

import altair as alt
alt.data_transformers.disable_max_rows()

from src.preprocess.preprocess import load_data
from src.plot.altair import plot_decompose, plot_all
from src.forecast.forecast import forecast

In [ ]:
boxid = [
    "ESD.000088-1",
    "063.623-1",
    "VRY.CHOPS-1",
    "HVT.111153-1",
    "TTR.251049-1",
    "BGL.CROLA-1",
][3]

## forecast

In [ ]:
%%time
df_total = forecast(boxid=boxid)

In [ ]:
plot_all(df_total, df_meta=None)

In [ ]:
len(df_total)

In [ ]:
plot_decompose(df_total)

## run all

In [ ]:
%%time

boxid = [
    "ESD.000088-1",
    "063.623-1",
    "VRY.CHOPS-1",
    "HVT.111153-1",
    "TTR.251049-1",
    "BGL.CROLA-1",
]

for box in boxid:
    df_total = forecast(boxid=box)
    
    df_total["processed_on"] = dt.datetime.now(tz=pytz.timezone("UTC"))
    df_total["is_valid"] = True

    con_config = format_connection("DALI_forecasts")

    with create_engine(URL(**con_config)).connect() as con:
        df_total.to_sql(
            name=con_config["table"],
            con=con,
            schema=con_config["schema"],
            if_exists="append",
            index=False,
        )

## write results

In [ ]:
import pandas as pd
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from src.utils.parser import parse_config
from src.utils.preprocess import downcast
from src.utils.vault import get_secrets
from src.utils.snowflake import format_connection

## write results

In [ ]:
import pytz

In [ ]:
# create or replace table DB_DATASCIENCE_P.DATASCIENCE_2_PUBLISH.DS_SPARK_DALI_WEEK_FORECASTS
# (
#     BOXID        VARCHAR(50),
#     DATE         TIMESTAMPNTZ,
#     L            VARCHAR(5),
#     PROCESSED_ON TIMESTAMPNTZ,
#     WEEK         NUMBER(2),
#     YEAR         NUMBER(4),
#     EXTREME      VARCHAR(3),
#     VALUE        DOUBLE,
#     PERIOD       VARCHAR(10),
#     MODEL_VAR    VARCHAR(10),
#     BAND         VARCHAR(10),
#     BOUNDARY     VARCHAR(10),
#     IS_VALID     BOOLEAN
# );

In [ ]:
df_total["processed_on"] = dt.datetime.now(tz=pytz.timezone("UTC"))
df_total["is_valid"] = True

con_config = format_connection("DALI_forecasts")

with create_engine(URL(**con_config)).connect() as con:
    df_total.to_sql(
        name=con_config["table"],
        con=con,
        schema=con_config["schema"],
        if_exists="append",
        index=False,
    )

In [ ]:
df_total